THOMAS MCDONNELL QUANT TRADING STRATEGIES HOMEWORK 3

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm
from pandas import DateOffset

import warnings

# Suppress the warning
warnings.filterwarnings("ignore")

Worked with Sean Lin

In [2]:
FR = pd.read_csv("C:/Users/thoma/Downloads/ZACKS_FR_2_f40c6a304f87d9f492c1f21839d474e2\ZACKS_FR_2_f40c6a304f87d9f492c1f21839d474e2.csv")

In [3]:
FR['per_end_date'] = pd.to_datetime(FR['per_end_date'])
start_date = datetime(2014, 6, 1)
FR = FR[FR['per_end_date'] >= start_date]
end_date = datetime(2022, 1, 31)
FR = FR[FR['per_end_date'] <= end_date]
FR = FR.set_index('ticker')

#Drop all of the tickers that don't have quarterly data
FR = FR[FR['per_type'] == 'Q']

In [4]:
FC = pd.read_csv("C:/Users/thoma\Downloads\ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b/ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b.csv")


In [5]:
FC = FC[['ticker','zacks_sector_code', 'per_end_date', 'filing_date']]

In [6]:
FC = FC.drop_duplicates(subset = 'ticker')

#Drop all tickers in the finance, industry and auto sector

FC = FC[~FC['zacks_sector_code'].isin([5.0, 13.0])]
FC = FC.set_index('ticker')

In [7]:
common_tickers = set(FC.index).intersection(set(FR.index))
FR = FR.loc[common_tickers]

In [8]:
FR['tot_debt_tot_equity'] = pd.to_numeric(FR['tot_debt_tot_equity'])

In [9]:
grouped = FR.groupby('ticker')

# Count the number of rows where tot_debt_tot_equity is greater than 0.1
counts = grouped['tot_debt_tot_equity'].apply(lambda x: (x > 0.1).sum())

# Filter out the tickers that don't have more than 85% of their rows with tot_debt_tot_equity greater than 0.1
FR = grouped.filter(lambda x: (x['tot_debt_tot_equity'] > 0.1).sum() > len(x)*.85)


In [10]:
FR = FR.reset_index()

In [11]:
FC = pd.read_csv("C:/Users/thoma\Downloads\ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b/ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b.csv")
FC = FC[['ticker', 'per_type','per_end_date', 'filing_date', 'tot_lterm_debt', 'net_lterm_debt', 'eps_diluted_net']]

In [12]:
FC = FC[FC['per_type'] == 'Q']

In [13]:
FC['per_end_date'] = pd.to_datetime(FC['per_end_date'])
FC['filing_date'] = pd.to_datetime(FC['filing_date'])
merged_df = pd.merge(FR, FC, on=['per_end_date', 'ticker'])
merged_df = merged_df.drop_duplicates()

In [14]:
merged_df = merged_df.set_index('ticker')

In [15]:
merged_df = merged_df.dropna(subset=['filing_date'])

In [16]:
unique_tickers = merged_df.index.unique()
unique_tickers

Index(['PSTV', 'KEYS', 'BGC', 'CST', 'FBIO', 'CTLP', 'PVG', 'CNX', 'PAY.',
       'ALYE',
       ...
       'FUL', 'ALB', 'GEVO', 'OVRL', 'GNCMA', 'ISCB.', 'CXDC', 'MVNR.', 'XRX',
       'SNDK'],
      dtype='object', name='ticker', length=1892)

In [17]:
ticker_counts = {}
for ticker in tqdm(unique_tickers):
    # Get the number of rows for the current ticker
    num_rows = merged_df.loc[merged_df.index == ticker].shape[0]
    # Add the ticker and number of rows to the dictionary
    ticker_counts[ticker] = num_rows


100%|██████████| 1892/1892 [00:02<00:00, 824.00it/s]


In [18]:
#drop all tickers where they dont have at least 31 instances of data
tickers_to_drop = [ticker for ticker in ticker_counts if ticker_counts[ticker] != 31]

# Drop rows where the 'ticker' column is in the list of tickers to drop
merged_df = merged_df[~merged_df.index.isin(tickers_to_drop)]
FR
unique_tickers = merged_df.index.unique().tolist()

In [19]:
tickers_to_remove = ['SWIOU', 'GEF.B' 'SENEB', 'BUKS', 'NUVR', 'INRD', 'TMS.', 'GBCS', 'UNIR', 'JANL', 'STCC', 'SMID', 'TPCS', 'TDSNA', 'BVHBB', 'BGSF']

In [20]:
tickers = [x for x in unique_tickers if x not in tickers_to_remove]

Now I have a list of tickers whose data I will be taking to calculate the financial ratios. All of the work below here is about pulling in ZACKs dataframes and extracting all of the data. 

For all ZACK tables that aren't FC I will also need to merge my data on the filing date again to ensure that I have a filing date

In [21]:
merged_df = merged_df.loc[tickers]

In [22]:
merged_df = merged_df[['per_end_date', 'filing_date', 'tot_debt_tot_equity', 'ret_invst', 'net_lterm_debt', 'tot_lterm_debt', 'eps_diluted_net']]

In [23]:
filtered_df = merged_df[merged_df.index.isin(tickers)]

to do tomorrow: add SHRS/shares_out, MKTV/MKT_VAL, deal with getting prices on per end dates, shift all of my data to us knowing it the day after the filing date instead of on the filing date, change all EPS to equal .001 if negative, if net_lterm_debt is all NA, drop the ticker, if only a few NA set to 0. 

In [24]:
filtered_df

,per_end_date,filing_date,tot_debt_tot_equity,ret_invst,net_lterm_debt,tot_lterm_debt,eps_diluted_net
ticker,,,,,,,
PSTV,2014-06-30,2014-08-11,24.7192,-57.1567,NaN,19.501,-1124.4380
PSTV,2014-09-30,2014-11-07,-8.6930,-55.0795,-1.303,20.332,-899.5502
PSTV,2014-12-31,2015-03-16,-4.9828,-46.6407,-1.962,18.041,-524.7374
PSTV,2015-03-31,2015-05-11,-1.2401,292.0724,NaN,15.677,-23913.0400
PSTV,2015-06-30,2015-08-10,8.1425,24.1271,-7.332,16.184,225.0000
...,...,...,...,...,...,...,...
XRX,2020-12-31,2021-02-25,0.7649,0.7809,133.000,4050.000,0.3500
XRX,2021-03-31,2021-04-30,0.7712,0.4186,-95.000,3674.000,0.1800
XRX,2021-06-30,2021-08-04,0.7695,0.9993,-209.000,3597.000,0.4600


Dealing with Shares

In [25]:
SHRS = pd.read_csv("C:/Users/thoma/Downloads/ZACKS_SHRS_2_99db6fa97ac677f3c0d45a9fa9a70196\ZACKS_SHRS_2_99db6fa97ac677f3c0d45a9fa9a70196.csv")
SHRS = SHRS[['ticker', 'per_end_date', 'shares_out']]

In [26]:
SHRS = SHRS.set_index('ticker')

Dealing with market value

In [27]:
MKTV = pd.read_csv("C:/Users/thoma/Downloads/ZACKS_MKTV_2_ecb7f768974bbdd26964caefe2fd0378\ZACKS_MKTV_2_ecb7f768974bbdd26964caefe2fd0378.csv")
MKTV = MKTV[['ticker', 'per_end_date', 'mkt_val']]

In [28]:
MKTV['per_end_date'] = pd.to_datetime(MKTV['per_end_date'])

In [29]:
filtered_df = pd.merge(filtered_df, MKTV, left_on = ['ticker', 'per_end_date'], right_on = ['ticker', 'per_end_date'])

Turning EPS all positive

In [30]:
filtered_df.loc[:, 'eps_diluted_net'] = filtered_df.loc[:, 'eps_diluted_net'].where(filtered_df.loc[:, 'eps_diluted_net'] >= 0, 0.001)

In [31]:
filtered_df = filtered_df.set_index('ticker')

In [32]:
filtered_df['filing_date'] = filtered_df['filing_date'] + DateOffset(days=1)

import price data

In [40]:
eod_price = pd.DataFrame()
for ticker in tqdm(unique_tickers):
    df =  quandl.get_table('QUOTEMEDIA/PRICES', ticker = [ticker], api_key = 'uoxc6V3s61czWuHoGLcs',
                    qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                    date = { 'gte': '2014-08-01', 'lte': '2022-02-01' }, 
                     paginate=True)
    eod_price = eod_price.append(df)
eod_price = eod_price.set_index('date')


100%|██████████| 1012/1012 [07:29<00:00,  2.25it/s]


In [46]:
eod_price

,ticker,adj_close
date,,
2022-02-01,PSTV,1.000000
2022-01-31,PSTV,0.944000
2022-01-28,PSTV,0.986500
2022-01-27,PSTV,0.916200
2022-01-26,PSTV,0.865700
...,...,...
2014-08-07,XRX,25.937491
2014-08-06,XRX,26.057851
2014-08-05,XRX,26.238390


In [47]:
eod_price = eod_price.sort_values(by='date')

In [48]:
filtered_df = filtered_df.sort_values(by = 'filing_date')
filtered_df['date'] = filtered_df['filing_date']

In [49]:
n =pd.merge_asof(eod_price, filtered_df, by = 'ticker', left_on = 'date', right_on = 'date')

In [50]:
n = n.set_index('ticker')

In [51]:
LLY = n.loc['LLY']

In [52]:
n = n.reset_index()

In [53]:
tickers = n['ticker'].unique().tolist()

In [54]:
n = n.set_index('ticker')

In [55]:
n['net_lterm_debt'] = n['net_lterm_debt'].fillna(n['tot_lterm_debt'])
n['net_lterm_debt'] = n['net_lterm_debt'].fillna(0)

In [75]:
for ticker in tqdm(n.index.unique()):
    if n.loc[ticker].isna().any().any():
        n = n.drop(ticker)

100%|██████████| 972/972 [03:18<00:00,  4.89it/s]


In [78]:
tickers = n.index.unique().tolist()

Loop to get my ratios

In [118]:
n = n.loc[tickers]

In [126]:
LLY = n.loc['LLY']
LLY.columns

Index(['date', 'adj_close', 'per_end_date', 'filing_date',
       'tot_debt_tot_equity', 'ret_invst', 'net_lterm_debt', 'tot_lterm_debt',
       'eps_diluted_net', 'mkt_val'],
      dtype='object')

In [165]:
f = pd.DataFrame()
for ticker in tqdm(tickers):
    df = n.loc[ticker]
    df = df.merge(df, left_on='per_end_date', right_on='date', how = 'outer').ffill()
    df = df.rename(columns = {'adj_close_y': 'per_end_price'})
    df['debt_to_market_cap'] = df['tot_debt_tot_equity_x']*df['per_end_price']/df['adj_close_x']
    df['price_to_earnings'] = df['adj_close_x']/df['eps_diluted_net_x']
    df['R'] = (df['ret_invst_x']*(df['net_lterm_debt_y'] + (df['mkt_val_x'])))
    df['ROI'] = df['R']/(df['net_lterm_debt_y']+df['mkt_val_x']*df['adj_close_x']/df['per_end_price']) 
    df['ticker'] = ticker
    df = df[['date_x', 'adj_close_x', 'debt_to_market_cap', 'price_to_earnings', 'ROI', 'ticker']]
    df = df.rename(columns = {'date_x': 'date', 'adj_close_x': 'price'})
    df = df.drop_duplicates(subset = 'date')
    f = df.append(f)

100%|██████████| 280/280 [00:04<00:00, 56.69it/s]


In [168]:
f = f.set_index('ticker')

KeyError: "None of ['ticker'] are in the columns"

In [169]:
f.to_csv('hw_3_ratio_data.csv')